# Exploration

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import re
import seaborn as sns
import weightedcalcs as wc

In [2]:
import sys
import os
sys.path.append('../src/features')
from build_features import fill_nulls, filter_col_with_regex, get_dummies

In [3]:
# read in data 
df = pd.read_csv('../data/raw/20161201_voter_study_group.csv')

In [5]:
df_no_nulls = fill_nulls(df)

In [8]:
calc = wc.Calculator('weight')
weight = df['weight']
d = calc.distribution(df_no_nulls, 'presvote16post_2016').round(3).sort_values(ascending=False)
d.count




presvote16post_2016
1.0    0.406
2.0    0.399
0.0    0.130
3.0    0.027
6.0    0.016
4.0    0.012
7.0    0.008
5.0    0.002
Name: weight, dtype: float64

## Community engagement features

In [ ]:
# return list of community engagement column names
col_list = []
for i in range(1, 18):
    col = f'volunteerorg2_{i}_baseline'
    col_list.append(col)
    
# other community engagement fields 
comm_list = ['CIVIC_PARTICIPATION_2016',
             'pew_churatd_2016',
             'volunteer_baseline',
             'volunteerf_baseline',
             'volunteernum_baseline'
            ]

df_predictors = df[(col_list + comm_list)]

# save target subset of dataframe
df_target = pd.DataFrame(df['presvote16post_2016'])

# predictors and target
df_target_predictors = df_target.join(df_predictors)

# correlation coefficients 
df_target_predictors.corr()['presvote16post_2016'].sort_values(ascending=False).head()

In [ ]:
sns.countplot(x=df_target.presvote16post_2016)
sns.xticklabels(df_target.columns)

In [ ]:
# fill null values 
# df = fill_nulls(df)

In [ ]:
# with open('../data/processed/predictors_target.text', 'wb') as f:
#     pickle.dump([X, y], f)